In [1]:
import argparse
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
from torch.autograd import Variable
import os
import sys
sys.path.insert(0, './../')
from experiment_abstract.dataset import Dictionary, VQAFeatureDataset
import model_predict as base_model
print('ok')
os.chdir('./../')
exp = './experiment_abstract/'


torch.manual_seed(1111)
torch.cuda.manual_seed(1111)
torch.backends.cudnn.benchmark = True


dictionary = Dictionary.load_from_file(exp+'data/dictionary.pkl')
train_dset = VQAFeatureDataset('train', dictionary, dataroot=exp+'data/')
eval_dset = VQAFeatureDataset('val', dictionary, dataroot=exp+'data/')

ok
loading dictionary from ./experiment_abstract/data/dictionary.pkl
loading features from h5 file
answer_path ./experiment_abstract/data/cache/train_target.pkl
loading features from h5 file
answer_path ./experiment_abstract/data/cache/val_target.pkl


In [2]:
batch_size = 1
num_hid = 1024

In [3]:
constructor = 'build_baseline0_newatt'
model = getattr(base_model, constructor)(train_dset, num_hid).cuda()
model.w_emb.init_embedding(exp+'data/glove6b_init_300d.npy')
train_loader = DataLoader(train_dset, batch_size, shuffle=True, num_workers=1)
eval_loader =  DataLoader(eval_dset, batch_size, shuffle=True, num_workers=1)

In [4]:
model_path = exp+'saved_models/my_model_2/model.pth'
model_params = torch.load(model_path)

In [5]:
model_params.keys()[:3]

['module.w_emb.emb.weight',
 'module.q_emb.rnn.weight_ih_l0',
 'module.q_emb.rnn.weight_hh_l0']

In [6]:
model = nn.DataParallel(model).cuda()
model.load_state_dict(model_params)

In [7]:
model.eval()
pass

In [8]:
from train import train

In [9]:
optim = train(model, train_loader, eval_loader, num_epochs=1, output=None)

train.py:65: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(model.parameters(), 0.25)


In [10]:
optim

Adamax (
Parameter Group 0
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.002
    weight_decay: 0
)

In [29]:
optim.param_groups[0]['params'][-4].shape

torch.Size([2048, 2048])

In [44]:
optim.param_groups[0]['params'][-4].sum(1).cpu().detach().numpy().astype(str).tolist()

['-1.22387778759',
 '-1.35555100441',
 '-1.50392580032',
 '-0.9804251194',
 '-3.51587128639',
 '-0.638019859791',
 '-1.16341245174',
 '-2.64550971985',
 '-1.94561386108',
 '-2.27987766266',
 '-2.58548855782',
 '-0.781428098679',
 '-2.62219548225',
 '-1.94366359711',
 '-1.85014247894',
 '-0.907570242882',
 '-2.70031499863',
 '-1.77837705612',
 '-1.27628314495',
 '-1.68866705894',
 '-0.0789267122746',
 '-2.93500161171',
 '-2.44145584106',
 '-0.92251598835',
 '-1.77682471275',
 '-0.598081171513',
 '-3.35028553009',
 '-0.641321718693',
 '0.0402461141348',
 '-3.14947891235',
 '-0.938118159771',
 '-0.95746088028',
 '-3.08350372314',
 '-2.85538101196',
 '-1.75016462803',
 '0.366221815348',
 '0.720648527145',
 '-1.56930220127',
 '-3.51477384567',
 '-0.992180943489',
 '0.230636894703',
 '-0.37549996376',
 '-0.681141853333',
 '-1.78006017208',
 '0.213765218854',
 '-3.03287601471',
 '-3.19481515884',
 '-2.38019156456',
 '-1.83098876476',
 '-0.88933044672',
 '-1.45618617535',
 '-1.42119121552',
 '

In [12]:
for group in optim.param_groups:
    for p in group['params']:
        if p.grad is not None:
            print(p.grad.shape)

torch.Size([6560, 300])
torch.Size([3072, 300])
torch.Size([3072, 1024])
torch.Size([3072])
torch.Size([3072])
torch.Size([1024])
torch.Size([])
torch.Size([1024, 2048])
torch.Size([1024])
torch.Size([])
torch.Size([1024, 1024])
torch.Size([1])
torch.Size([])
torch.Size([1, 1024])
torch.Size([1024])
torch.Size([])
torch.Size([1024, 1024])
torch.Size([1024])
torch.Size([])
torch.Size([1024, 2048])
torch.Size([1024])
torch.Size([])
torch.Size([1024, 2048])
torch.Size([2048])
torch.Size([])
torch.Size([2048, 2048])
torch.Size([426])
torch.Size([])
torch.Size([426, 2048])


In [12]:
model.module.classifier.main

Sequential(
  (0): Linear(in_features=2048, out_features=2048, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.5, inplace)
  (3): Linear(in_features=2048, out_features=426, bias=True)
)